In [1]:
import os
import glob
import random
import shutil
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import models, layers
from tqdm import tqdm



In [2]:
!rm -rf dataset

# where the text files are going to live
dataset_path = "dataset"
dataset_path_all = os.path.join(dataset_path, "all")
dataset_path_train = os.path.join(dataset_path, "train")
dataset_path_valid = os.path.join(dataset_path, "valid")

# just use 20 files
file_number = 20

# gather the corpus if it has not been gathered yet
if not os.path.exists(dataset_path):

  # create all folders
  for path in [dataset_path, dataset_path_all, dataset_path_train, dataset_path_valid]:
    if not os.path.exists(path):
      os.mkdir(path)

  # clone the repo
  !git clone https://github.com/vilmibm/lovecraftcorpus

  # find all the files
  paths_all = glob.glob(os.path.join("lovecraftcorpus/*.txt"))

  # do not use all
  paths_all = paths_all[:file_number]

  # split 80/20
  split_index = int(len(paths_all) * 0.8)
  paths_train = paths_all[:split_index]
  paths_valid = paths_all[split_index:]

   # comment files
  def lilla_copy(paths, destination):
    for path in paths:
      shutil.copy2(path, destination)
  lilla_copy(paths_all, dataset_path_all)
  lilla_copy(paths_train, dataset_path_train)
  lilla_copy(paths_valid, dataset_path_valid)

  # delete repo
  !rm -rf lovecraftcorpus

  # done
  print("Corpus downloaded.")


Cloning into 'lovecraftcorpus'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (70/70), done.
Corpus downloaded.


In [3]:
batch_size = 32
seed = 42

def create_dataset(dataset_path):
  dataset = preprocessing.text_dataset_from_directory(
      dataset_path,
      labels=None,
      batch_size=batch_size,
      seed=seed
  )
  return dataset



dataset_original_all = create_dataset(dataset_path_all)
dataset_original_train = create_dataset(dataset_path_train)
dataset_original_valid = create_dataset(dataset_path_valid)


AttributeError: module 'tensorflow._api.v1.keras.preprocessing' has no attribute 'text_dataset_from_directory'

In [ ]:
for batch in dataset_original_all:
  for sample in batch[:4]:
    sample = sample.numpy()
    print(sample[:200])

b'THE DUNWICH HORROR\n\nGorgons and Hydras, and Chimaeras--dire stories of Celaeno and the Harpies--may reproduce themselves in the brain of superstition--but they were there before. They are transcripts,'
b'THROUGH THE GATES OF THE SILVER KEY\n\nChapter One\n\nIn a vast room hung with strangely figured arras and carpeted with Bonkhata rugs of impressive age and workmanship, four men were sitting around a doc'
b'THE STREET\n\nMen of strength and honour fashioned that Street: good valiant men of our blood who had come from the Blessed Isles across the sea. At first it was but a path trodden by bearers of water f'
b'AT THE MOUNTAINS OF MADNESS\n\nI\n\nI am forced into speech because men of science have refused to follow my advice without knowing why. It is altogether against my will that I tell my reasons for opposin'


2021-11-06 14:38:40.137695: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-06 14:38:40.139323: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
vocabulary_size = 10000
encoder = layers.TextVectorization(
    max_tokens=vocabulary_size,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    pad_to_max_tokens="multi_hot",
    output_mode="int"
)
# learning
encoder.adapt(dataset_original_all)

vocabulary = encoder.get_vocabulary()
print(f"Vocabulary size: {len(vocabulary)}")
print(f"Vocabulary: {vocabulary}")


Vocabulary size: 10000
Vocabulary: ['', '[UNK]', 'the', 'of', 'and', 'to', 'a', 'in', 'was', 'i', 'that', 'had', 'it', 'as', 'he', 'with', 'on', 'at', 'which', 'from', 'not', 'for', 'but', 'were', 'his', 'we', 'my', 'an', 'by', 'all', 'they', 'be', 'this', 'or', 'there', 'have', 'could', 'one', 'been', 'me', 'no', 'when', 'some', 'what', 'their', 'our', 'would', 'is', 'its', 'so', 'more', 'old', 'him', 'only', 'out', 'now', 'did', 'them', 'about', 'into', 'up', 'than', 'seemed', 'then', 'very', 'before', 'any', 'after', 'like', 'though', 'those', 'even', 'through', 'great', 'must', 'who', 'things', 'time', 'over', 'other', 'saw', 'down', 'if', 'where', 'us', 'made', 'came', 'found', 'thing', 'men', 'these', 'such', 'never', 'might', 'most', 'many', 'still', 'man', 'much', 'first', 'whose', 'strange', 'certain', 'place', 'yet', 'come', 'beyond', 'back', 'thought', 'upon', 'years', 'too', 'two', 'night', 'almost', 'new', 'long', 'knew', 'you', 'ancient', 'without', 'told', 'seen', 'are',

2021-11-06 14:38:47.921967: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
sequence_length = 32

def create_dataset_for_autoregression(dataset):
  x_inputs = []
  y_outputs = []
  for batch in dataset:
    batch = encoder(batch).numpy()
    for sample in tqdm(batch):
        # pad at the beginning
        padding_token_id = 0
        padding = [padding_token_id] * sequence_length
        sample = padding + list(sample)

        # map all to input output pairs
        for start_index in range(0, len(sample) - sequence_length):
          x = sample[start_index:start_index + sequence_length]
          y = sample[start_index + sequence_length]
          if y == 0:
            break
          x_inputs += [x]
          y_outputs += [y]

  return tf.data.Dataset.from_tensor_slices((x_inputs, y_outputs))

dataset_train = create_dataset_for_autoregression(dataset_original_train)
dataset_valid = create_dataset_for_autoregression(dataset_original_valid)

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 65.66it/s]


In [ ]:
embedding_size = 128

model = models.Sequential()

#don't need textvectorizer anymore
model.add(layers.Embedding(vocabulary_size, embedding_size, input_length=sequence_length))
model.add(layers.LSTM(512, return_sequences=True))
model.add(layers.Dropout(0.4))
model.add(layers.LSTM(1024))#should be bigger than the first LSTM
model.add(layers.Dropout(0.5)) #grows from layer to layer
model.add(layers.Dense(vocabulary_size, activation="softmax")) # classifier that picks one word from 

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    dataset_train.shuffle(10000).batch(1024), # batch size for speeding up things and rendering?
    validation_data=dataset_valid.batch(1024),
    epochs=10
)

render_history(history)

model.summary()

Epoch 1/10
